<a href="https://colab.research.google.com/github/ishanp404/mlprojects/blob/main/Road_Sign_Detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from keras.models import Sequential
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import os
from keras.layers import Dropout, Conv2D, MaxPool2D, Dense, Activation, Flatten

In [3]:
def load_and_preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image / 255.0
    return image


In [4]:
image_directory = '/content/drive/MyDrive/Road Sign/images'
annotation_directory = '/content/drive/MyDrive/Road Sign/annotations'

image_files = [f for f in os.listdir(image_directory) if f.endswith('.png')]

In [5]:
image_annotation_pairs = []
for image_filename in image_files:
    image_path = os.path.join(image_directory, image_filename)
    annotation_path = os.path.join(annotation_directory, image_filename.replace('.png', '.xml'))

    if os.path.exists(annotation_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        annotations = []
        for object_elem in root.findall('object'):
            obj_info = {
                'class': object_elem.find('name').text,
                'xmin': int(object_elem.find('bndbox/xmin').text),
                'ymin': int(object_elem.find('bndbox/ymin').text),
                'xmax': int(object_elem.find('bndbox/xmax').text),
                'ymax': int(object_elem.find('bndbox/ymax').text),
            }
            annotations.append(obj_info)

        image_annotation_pairs.append({'image': image, 'annotations': annotations})

In [6]:
class_mapping = {
    'trafficlight': 0,
    'stop': 1,
    'speedlimit': 2,
    'crosswalk': 3
}

for data_point in image_annotation_pairs:
    annotations = data_point['annotations']
    for annotation in annotations:
        class_name = annotation['class']
        if class_name in class_mapping:
            annotation['class'] = class_mapping[class_name]
        else:
            annotation['class'] = -1
    annotations = data_point['annotations']
    class_labels = [annotation['class'] for annotation in annotations]
    data_point['annotations'] = class_labels[0]

In [7]:
import random

random.seed(42)

random.shuffle(image_annotation_pairs)

split_ratio = 0.8
split_index = int(len(image_annotation_pairs) * split_ratio)

training_data = image_annotation_pairs[:split_index]
testing_data = image_annotation_pairs[split_index:]

In [8]:
x_train = [data_point['image'] for data_point in training_data]
y_train = [data_point['annotations'] for data_point in training_data]

x_test = [data_point['image'] for data_point in testing_data]
y_test = [data_point['annotations'] for data_point in testing_data]

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train, dtype=np.int64)
y_test = np.array(y_test, dtype=np.int64)

x_train = x_train/255.0
x_test = x_test/255.0

In [9]:
x_train.shape

(701, 224, 224, 3)

In [10]:
x_test.shape

(176, 224, 224, 3)

In [70]:
MODEL_URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5'

In [71]:
INPUT_SHAPE = (None, 224, 224, 3)

In [87]:
from keras.layers import Conv2D, MaxPool2D, Dense, Activation, Dropout, Flatten
import tensorflow_hub as hub

model = Sequential([
    hub.KerasLayer(MODEL_URL)
])
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["sparse_categorical_accuracy"]
)

In [88]:
model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'], metrics=['accuracy'])

In [89]:
model.build(input_shape=INPUT_SHAPE)

In [90]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_17 (KerasLayer  (None, 1001)              5432713   
 )                                                               
                                                                 
 flatten_10 (Flatten)        (None, 1001)              0         
                                                                 
 dense_18 (Dense)            (None, 128)               128256    
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 4)                 516       
                                                     

In [91]:
from keras.callbacks import EarlyStopping
ec = EarlyStopping(monitor='val_loss', patience=20)

In [92]:
model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=30, callbacks=[ec])

Epoch 1/30
22/22 [==============================] - 8s 130ms/step - loss: 1.1451 - accuracy: 0.7247 - val_loss: 0.4325 - val_accuracy: 0.8523
Epoch 2/30
22/22 [==============================] - 2s 70ms/step - loss: 0.3747 - accuracy: 0.8887 - val_loss: 0.3785 - val_accuracy: 0.8580
Epoch 3/30
22/22 [==============================] - 2s 71ms/step - loss: 0.3066 - accuracy: 0.9001 - val_loss: 0.3450 - val_accuracy: 0.8807
Epoch 4/30
22/22 [==============================] - 2s 70ms/step - loss: 0.2125 - accuracy: 0.9330 - val_loss: 0.3230 - val_accuracy: 0.8920
Epoch 5/30
22/22 [==============================] - 2s 70ms/step - loss: 0.1731 - accuracy: 0.9458 - val_loss: 0.3280 - val_accuracy: 0.8807
Epoch 6/30
22/22 [==============================] - 2s 70ms/step - loss: 0.1450 - accuracy: 0.9586 - val_loss: 0.3694 - val_accuracy: 0.8864
Epoch 7/30
22/22 [==============================] - 2s 70ms/step - loss: 0.1379 - accuracy: 0.9501 - val_loss: 0.3792 - val_accuracy: 0.8920
Epoch 8/30
2

In [93]:
model.evaluate(x_test,y_test)

6/6 [==============================] - 0s 47ms/step - loss: 0.4887 - accuracy: 0.8920


[0.4886554777622223, 0.8920454382896423]